In [3]:
# Filter Checking for FILL files

import ROOT as r
from collections import Counter
import math

filename = "00289231_00000001_1.highstats-Small-B2CC-UP.root"
treename = "ST-b2cc"

file = r.TFile.Open(filename)
tree = file.Get(treename)

# Example bad run list
bad_runs = [303681, 304475, 304487, 304488, 304489, 304490, 304491]

print(f"Checking file {filename}")

# Check number of entries (should be > 20000)
no_entries = tree.GetEntries()
if no_entries > 20000:
    print (f"Number of entries = {no_entries}: enough :)")
else:
    print(f"Number of entries = {no_entries}: bad :(")

# Check if any bad runs
for bad_run in bad_runs:
    n = tree.GetEntries(f"run == {bad_run}")
    if n > 0:
        print(f"Number of entries with run {bad_run}: {n}")

# Check if unique block
block_values = set()
for i in range(tree.GetEntries()):
    tree.GetEntry(i)
    block_values.add(tree.block)
    if len(block_values) > 1:
        break

print(f"Unique block values in the file: {block_values}")

# Check if unique fill
fill_values = set()
for i in range(tree.GetEntries()):
    tree.GetEntry(i)
    fill_values.add(tree.FILL)
    if len(fill_values) > 1:
        break

print(f"Unique fill values in the file: {fill_values}")

# Check duplicates
event_ids = []
for i in range(tree.GetEntries()):
    tree.GetEntry(i)
    event_ids.append(tree.event)  # Replace with correct branch name if needed

counts = Counter(event_ids)
num_duplicates = sum(1 for count in counts.values() if count > 1)
fraction_duplicates = num_duplicates / tree.GetEntries()

print(f"Fraction of duplicated events: {fraction_duplicates}")

# Check for NaN or invalid values
nan_count = 0
for i in range(tree.GetEntries()):
    tree.GetEntry(i)
    if (
        not isinstance(tree.event, (int, float)) or math.isnan(tree.event) or
        not isinstance(tree.Bp_DTF_OwnPV_MASS, (int, float)) or math.isnan(tree.Bp_DTF_OwnPV_MASS) or
        not isinstance(tree.FILL, (int, float)) or math.isnan(tree.FILL) or
        not isinstance(tree.run, (int, float)) or math.isnan(tree.run)
    ):
        nan_count += 1

print(f"Number of entries with NaN or non-numeric values in event/dtf_m/FILL/run: {nan_count}")

Checking file 00289231_00000001_1.highstats-Small-B2CC-UP.root
Number of entries = 21983084: enough :)
Unique block values in the file: {8}
Unique fill values in the file: {10232.0, 10230.0}
Fraction of duplicated events: 0.13196506004344066
Number of entries with NaN or non-numeric values in event/dtf_m/FILL/run: 0


In [ ]:
#Check duplicate distribution
from collections import Counter

filename = "00289231_00000001_1.highstats-Small-B2CC-UP.root"
treename = "ST-b2cc"

file = r.TFile.Open(filename)
tree = file.Get(treename)

event_ids = []
for i in range(tree.GetEntries()):
    tree.GetEntry(i)
    event_ids.append((tree.run, tree.event))  # use (run, event) for better uniqueness

counts = Counter(event_ids)
duplicates = {eid: count for eid, count in counts.items() if count > 1}
print(f"{len(duplicates)} duplicated event IDs (out of {len(counts)} unique events)")


In [4]:
import ROOT

# Enable ROOT graphics in Jupyter/SWAN
%jsroot on

# Open the ROOT file
file = ROOT.TFile("00289229_00000003_1.highstats-Small-B2CC-UP.root")

# Access the tree
tree = file.Get("ST-b2cc")

# Create histogram
hist = ROOT.TH1F("Bp_P_hist", "B^{+} Momentum;Bp_P [MeV/c];Events", 100, 0, 1e6)

# Fill histogram with the Bp_P variable
tree.Draw("Bp_P >> Bp_P_hist")

# Draw the histogram
canvas = ROOT.TCanvas()
hist.Draw()
canvas.Draw()


Info in <TCanvas::MakeDefCanvas>:  created default TCanvas with name c1


In [5]:
import ROOT
import numpy as np

# === Parameters ===
filename = "00289229_00000003_1.highstats-Small-B2CC-UP.root"
treename = "ST-b2cc"
momentum_var = "Bp_P"
bin_threshold = 0.05

# === Load TTree ===
file = ROOT.TFile.Open(filename)
tree = file.Get(treename)

# === Prepare momentum buffer ===
momentum_buffer = np.zeros(1, dtype=np.float32)
tree.SetBranchAddress(momentum_var, momentum_buffer)

# === Extract momentum values ===
momentum_vals = []
n_entries = tree.GetEntries()

for i in range(n_entries):
    tree.GetEntry(i)
    p = momentum_buffer[0]
    momentum_vals.append((i, p))

momentum_vals = np.array(momentum_vals)

# === Filter duplicates by binning ===
def filter_indices_by_binning(momentum_vals, threshold=0.05):
    values = momentum_vals[:, 1]
    bins = (values / (1 + threshold)).astype(int)
    _, unique_indices = np.unique(bins, return_index=True)
    return momentum_vals[unique_indices, 0].astype(int)

kept_indices = filter_indices_by_binning(momentum_vals, bin_threshold)

# === Save filtered tree ===
output_file = ROOT.TFile("filtered_" + filename, "RECREATE")
output_tree = tree.CloneTree(0)

for i in kept_indices:
    tree.GetEntry(int(i))
    output_tree.Fill()


output_tree.Write()
output_file.Close()

print(f"Filtered tree written to: filtered_{filename}")
print(f"Original entries: {n_entries}, Kept: {len(kept_indices)}")


KeyboardInterrupt: 

In [ ]:
from array import array
import ROOT

file = ROOT.TFile.Open("00289229_00000003_1.highstats-Small-B2CC-UP.root")
tree = file.Get("ST-b2cc")

event_entry_list = [962752, 763844, 849562, 417986, 560540]

entries_list = []
for entries in event_entry_list:
    tree.GetEntry(entries)
    entries_list.append(tree.event)

print(entries_list)
Bp_P_list = []

In [6]:
# Find event value that has atleast 5 duplicates

import ROOT
from collections import Counter

# File and tree
file_path = "00289229_00000003_1.highstats-Small-B2CC-UP.root"
tree_name = "ST-b2cc"

# Open file and tree
file = ROOT.TFile.Open(file_path)
tree = file.Get(tree_name)

# Collect all event values (assumes 'event' is the relevant identifier)
event_ids = []

for entry in tree:
    event_ids.append(entry.event)  # Modify 'event' if your branch name differs

# Count duplicatesg
event_counts = Counter(event_ids)
duplicates = {event: count for event, count in event_counts.items() if count >= 5}

# Output result
print(f"Number of event IDs with 5 or more duplicates: {len(duplicates)}")
for event, count in duplicates.items():
    print(f"Event ID: {event} - Count: {count}")


Number of event IDs with 5 or more duplicates: 21365
Event ID: 1571277867 - Count: 5
Event ID: 45107879259 - Count: 6
Event ID: 49813314880 - Count: 6
Event ID: 406291873 - Count: 5
Event ID: 21745053798 - Count: 5
Event ID: 4687316827 - Count: 5
Event ID: 87474094672 - Count: 9
Event ID: 23176201238 - Count: 5
Event ID: 54865212329 - Count: 5
Event ID: 7828747245 - Count: 6
Event ID: 29350598869 - Count: 6
Event ID: 1788185462 - Count: 7
Event ID: 13483828035 - Count: 7
Event ID: 94949978241 - Count: 22
Event ID: 1121654010 - Count: 5
Event ID: 78972544139 - Count: 5
Event ID: 85126413228 - Count: 7
Event ID: 13593803702 - Count: 9
Event ID: 23452471748 - Count: 5
Event ID: 9941226593 - Count: 5
Event ID: 10370231212 - Count: 8
Event ID: 18870472216 - Count: 6
Event ID: 2579773599 - Count: 5
Event ID: 30020973771 - Count: 6
Event ID: 51586569561 - Count: 5
Event ID: 39922662511 - Count: 6
Event ID: 23085042406 - Count: 5
Event ID: 50968509533 - Count: 5
Event ID: 38982462396 - Count: 

In [9]:
#Check Bp_P of duplicates for selected events

from array import array
import ROOT

target_events = [1121654010, 19726357590, 25355205984, 94949978241, 25078153653]  # your known value

for target_event in target_events:
    file = ROOT.TFile.Open("00289229_00000003_1.highstats-Small-B2CC-UP.root")
    tree = file.Get("ST-b2cc")

    event_buf = array('Q', [0])  # 'Q' = unsigned 64-bit integer (ULong64_t)
    tree.SetBranchAddress("event", event_buf)

    matching_entries = []

    for i in range(tree.GetEntries()):
        tree.GetEntry(i)
        if event_buf[0] == target_event:
            matching_entries.append(i)

    print(f"Found {len(matching_entries)} matching entries for event {target_event}:")
    print(matching_entries)
    
    Bp_P_list = []
    for matching_entry in matching_entries:
        tree.GetEntry(matching_entry)
        Bp_P_list.append(tree.Bp_P)

    print(f"Bp_P values for event {target_event}:")
    print(Bp_P_list)


Found 5 matching entries for event 1121654010:
[9176, 9177, 9178, 9179, 9180]
Bp_P values for event 1121654010:
[139387.8125, 135842.546875, 119004.640625, 111138.6875, 138274.65625]
Found 12 matching entries for event 19726357590:
[34934, 34935, 34936, 34937, 34938, 34939, 34940, 34941, 34942, 34943, 34944, 34945]
Bp_P values for event 19726357590:
[127264.421875, 121160.2578125, 127050.9453125, 120945.6796875, 121916.0703125, 115807.109375, 121702.2890625, 115592.078125, 122707.90625, 122491.2265625, 117349.453125, 117132.59375]
Found 2 matching entries for event 25355205984:
[849561, 849562]
Bp_P values for event 25355205984:
[56450.20703125, 56154.4921875]
Found 22 matching entries for event 94949978241:
[9121, 9122, 9123, 9124, 9125, 9126, 9127, 9128, 9129, 9130, 9131, 9132, 9133, 9134, 9135, 9136, 9137, 9138, 9139, 9140, 9141, 9142]
Bp_P values for event 94949978241:
[144972.828125, 137760.609375, 145576.84375, 197984.53125, 145600.78125, 277285.375, 144411.90625, 145018.546875, 

event at entry 962752: 80394387231


In [ ]:
import ROOT
import numpy as np
import random
from collections import defaultdict
from array import array

# --- Load tree ---
filename = "00289229_00000003_1.highstats-Small-B2CC-UP.root"
treename = "ST-b2cc"
momentum_var = "Bp_P"
event_var = "eventNumber"

file = ROOT.TFile.Open(filename)
tree = file.Get(treename)

# --- Buffers ---
momentum_buf = array('f', [0])  # 'f' = float32
event_buf = array('L', [0])     # 'L' = unsigned long (works for ULong64_t)

tree.SetBranchAddress(momentum_var, momentum_buf)
tree.SetBranchAddress(event_var, event_buf)

n_entries = tree.GetEntries()

# --- Step 1: Pick 5 random entries and store their event numbers ---
random_indices = random.sample(range(n_entries), 5)
target_events = set()

for idx in random_indices:
    tree.GetEntry(idx)
    target_events.add(event_buf[0])

print(f"Selected eventNumbers: {sorted(target_events)}")

# --- Step 2: Collect Bp_P for all entries with matching eventNumber ---
event_to_momenta = defaultdict(list)
event_to_indices = defaultdict(list)

for i in range(n_entries):
    tree.GetEntry(i)
    ev_id = event_buf[0]
    if ev_id in target_events:
        p = momentum_buf[0]
        event_to_momenta[ev_id].append(p)
        event_to_indices[ev_id].append(i)

# --- Step 3: Display results ---
for ev_id in sorted(event_to_momenta.keys()):
    print(f"\nEvent {ev_id} has {len(event_to_momenta[ev_id])} entries:")
    for j, p in enumerate(event_to_momenta[ev_id]):
        print(f"  Entry {event_to_indices[ev_id][j]}: Bp_P = {p:.2f}")


In [2]:
print("hello")

helloasc
